# Detecting TRY mentions in sentences

In [1]:
import pandas as pd
import numpy as np
import spacy
df_dev=pd.read_pickle("../ProcessedData/procon_corpus.pkl")
df_dev=df_dev.loc[:,"chem_word":"label"]
df_dev.head()

,chem_word,prod_word,chem_word_idx,prod_word_idx,sentence,tokens,between_tokens,chems,prods,chem_pos_word,label
0,alpha-D-glucose,0.2mM,"[5, 10]","[3, 4]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...",[added],"[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph...",-1
1,alpha-D-glucose,25mM,"[5, 10]","[29, 30]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...","[(, 0.4mM, alpha, -, D, -, xylose, ), , incre...","[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph...",-1
2,alpha-D-xylose,0.2mM,"[12, 17]","[3, 4]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...","[added, alpha, -, D, -, glucose, (, 0.4mM]","[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph...",-1
3,alpha-D-xylose,25mM,"[12, 17]","[29, 30]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...","[), , increased, the, rate, of, enzymically, ...","[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph...",-1
4,fluoride,0.2mM,"[27, 28]","[3, 4]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...","[added, alpha, -, D, -, glucose, (, 0.4mM, alp...","[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph...",-1


In [2]:
import pandas as pd
import numpy as np
df_train = pd.read_pickle("../ProcessedData/train_corpus.pkl")
df_train = df_train.loc[:,"chem_word":"label"]
df_train.head()

,chem_word,prod_word,chem_word_idx,prod_word_idx,sentence,tokens,between_tokens,chems,prods,chem_pos_word,label
0,icariside D2,3.80 g/L,"[22, 24]","[17, 21]",Under the optimal conditions in fed-batch shak...,"[Under, the, optimal, conditions, in, fed, -, ...",[of],"[25*#*26, 41*#*43, 44*#*47, 28*#*29, 22*#*24]","[36*#*40, 17*#*21]","{'22*#*24': 'icariside D2', '41*#*43': 'icaris...",1
1,icariside D2,2.92 g/L,"[41, 43]","[36, 40]",Under the optimal conditions in fed-batch shak...,"[Under, the, optimal, conditions, in, fed, -, ...",[of],"[25*#*26, 41*#*43, 44*#*47, 28*#*29, 22*#*24]","[36*#*40, 17*#*21]","{'22*#*24': 'icariside D2', '41*#*43': 'icaris...",1
2,icariside D2,2.92 g/L,"[22, 24]","[36, 40]",Under the optimal conditions in fed-batch shak...,"[Under, the, optimal, conditions, in, fed, -, ...","[using, glucose, as, sole, carbon, source, ,, ...","[25*#*26, 41*#*43, 44*#*47, 28*#*29, 22*#*24]","[36*#*40, 17*#*21]","{'22*#*24': 'icariside D2', '41*#*43': 'icaris...",-1
3,glucose,3.80 g/L,"[25, 26]","[17, 21]",Under the optimal conditions in fed-batch shak...,"[Under, the, optimal, conditions, in, fed, -, ...","[of, icariside, D2, using]","[25*#*26, 41*#*43, 44*#*47, 28*#*29, 22*#*24]","[36*#*40, 17*#*21]","{'22*#*24': 'icariside D2', '41*#*43': 'icaris...",-1
4,glucose,2.92 g/L,"[25, 26]","[36, 40]",Under the optimal conditions in fed-batch shak...,"[Under, the, optimal, conditions, in, fed, -, ...","[as, sole, carbon, source, ,, and, the, BMT23-...","[25*#*26, 41*#*43, 44*#*47, 28*#*29, 22*#*24]","[36*#*40, 17*#*21]","{'22*#*24': 'icariside D2', '41*#*43': 'icaris...",-1


In [3]:
df_concat = pd.concat([df_dev, df_train], axis=0)
df_concat = df_concat.reset_index(drop=True)
df_concat.to_pickle("../ProcessedData/train_procon_corpus.pkl")  #2063

In [4]:
import pandas as pd
import numpy as np
import spacy
treenlp = spacy.load("en_core_web_sm")
df_dev=pd.read_pickle("../ProcessedData/train_procon_corpus.pkl")
new_df_dev = df_dev["label"]
labels = [1 if i == 1 else 0 for i in new_df_dev]
Y_dev = np.array(labels)
df_dev=df_dev.loc[:,"chem_word":"chem_pos_word"]
df_dev.head()

,chem_word,prod_word,chem_word_idx,prod_word_idx,sentence,tokens,between_tokens,chems,prods,chem_pos_word
0,alpha-D-glucose,0.2mM,"[5, 10]","[3, 4]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...",[added],"[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph..."
1,alpha-D-glucose,25mM,"[5, 10]","[29, 30]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...","[(, 0.4mM, alpha, -, D, -, xylose, ), , incre...","[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph..."
2,alpha-D-xylose,0.2mM,"[12, 17]","[3, 4]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...","[added, alpha, -, D, -, glucose, (, 0.4mM]","[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph..."
3,alpha-D-xylose,25mM,"[12, 17]","[29, 30]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...","[), , increased, the, rate, of, enzymically, ...","[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph..."
4,fluoride,0.2mM,"[27, 28]","[3, 4]",As as 0.2mM added alpha-D-glucose (0.4mM alph...,"[As, , as, 0.2mM, added, alpha, -, D, -, gluc...","[added, alpha, -, D, -, glucose, (, 0.4mM, alp...","[27*#*28, 12*#*17, 5*#*10, 30*#*36]","[11*#*12, 3*#*4, 29*#*30]","{'5*#*10': 'alpha-D-glucose', '12*#*17': 'alph..."


In [5]:
df_dev.shape

(2063, 10)

In [6]:
def get_sentence_lable(cand): 
    tokens = cand.tokens 
    sentence_positive_tokens =  {"batch","pathway",'fermentation','production','engineer','yield','titer',\
                  'biosynthesis','synthesis','flask','heterologous','titre','yields','concentration','incubation',\
                 'metabolic','novo','coli','accumulation','conversion','converted','productivity','metabolites',\
                 'productivities','quantities','produced','convert','flasks','laboratory','Yields','growth',\
                 'reaching','detected','content','concentrations','reached','synthesized','producing','accumulated',\
                 'resulting','accumulate','fermenter','contents','titers','rate','biotransformation','produces',\
                                'Increasing','enhances','produces'}
    if len(sentence_positive_tokens.intersection(set(tokens)))>0:
        lable = 1
    else:
        lable = 0
    return lable

In [7]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1
def get_pos_in_order(chemprod_list):
    """
    #input = [11*#*14, 20*#*21, 27*#*28, 9*#*10, 34*#*35]
    #output = [[9, 10], [11, 14], [20, 21], [27, 28], [34, 35]]
    """
    chemprod_pos_list = [[int(chemprod_pos) for chemprod_pos in chemprod.split("*#*")] \
                         for chemprod in chemprod_list]
    chemprod_pos_order_list = (sorted(chemprod_pos_list, key=lambda k: k[0]))
    return chemprod_pos_order_list

def get_prod_chem_span(x):
    """
    prod_left,prod_right,chem_left,chem_right
    span_pos = [prod_left,chem_right]
    """
    chems = x.chems
    prods = x.prods
    ProdChemBetweenOne = {'of',')','-'}
    chem_pos_order = get_pos_in_order(chems)
    prod_pos_order = get_pos_in_order(prods)
    
    #print(chem_pos_order,prod_pos_order)
    prod_chem_spans = []
    prod_chem_span_left_edge = {}
    prod_chem_span_right_edge = {}
    for prod_pos in prod_pos_order:
        prod_left ,prod_right = prod_pos
        for chem_pos in chem_pos_order:
            chem_left,chem_right = chem_pos
            chem_pos_diff = chem_left - prod_right
            if chem_pos_diff <0:
                continue
            elif chem_pos_diff == 0:
                prod_chem = [prod_left,chem_right]
                prod_chem_spans.append(prod_chem)
                prod_chem_span_right_edge[prod_left] = chem_right
                prod_chem_span_left_edge[chem_right] = prod_left
                break
            elif chem_pos_diff == 1: 
                between_tokens = x.tokens[prod_right:chem_left]
                if len(ProdChemBetweenOne.intersection(set(between_tokens))) > 0:
                    prod_chem = [prod_left,chem_right]
                    prod_chem_spans.append(prod_chem)
                    prod_chem_span_right_edge[prod_left] = chem_right
                    prod_chem_span_left_edge[chem_right] = prod_left                    
                    break
            else:
                break
    return prod_chem_spans,prod_chem_span_left_edge,prod_chem_span_right_edge

In [8]:
def get_clear_prodpos_chemtext(x):
    chems = x.chems
    prods = x.prods
    chem_pos_word = x.chem_pos_word
    ProdChemBetweenOne = {'of',')','-'}
    chem_pos_order = get_pos_in_order(chems)
    prod_pos_order = get_pos_in_order(prods)
    #input = [11*#*14, 20*#*21, 27*#*28, 9*#*10, 34*#*35]
    #output = [[9, 10], [11, 14], [20, 21], [27, 28], [34, 35]]    
    prodpos_chemtext = {}
    for prod_pos in prod_pos_order:
        prod_left ,prod_right = prod_pos
        for chem_pos in chem_pos_order:
            chem_left,chem_right = chem_pos
            chem_pos_diff = chem_left - prod_right
            if chem_pos_diff <0:
                continue
            elif chem_pos_diff == 0:
                prod_clear_pos = prod_pos
                prod_pos = "{}*#*{}".format(prod_left ,prod_right)
                chem_pos = "{}*#*{}".format(chem_left,chem_right)
                chem_clear_text = chem_pos_word[chem_pos]
                prodpos_chemtext[prod_pos] = chem_clear_text
            elif chem_pos_diff == 1:
                between_tokens = x.tokens[prod_right:chem_left]
                if len(ProdChemBetweenOne.intersection(set(between_tokens))) > 0:
                    prod_pos = "{}*#*{}".format(prod_left ,prod_right)
                    
                    prod_clear_pos = prod_pos
                    chem_pos = "{}*#*{}".format(chem_left,chem_right)
                    chem_clear_text = chem_pos_word[chem_pos]
                    #print("prod_pos",prod_pos,chem_clear_text)
                    prodpos_chemtext[prod_pos] = chem_clear_text  
            else:
                break
    chem_prod_between_tokens = {'(','of','concentration','concentrations','were','was',',',';'}
    for prod_pos in prod_pos_order:
        prod_left ,prod_right = prod_pos
        for chem_pos in chem_pos_order:
            chem_left,chem_right = chem_pos
            chem_pos_diff = prod_left - chem_right
            #print("3"*30,chem_pos_diff)
            
            if chem_pos_diff >2:
                
                continue
            elif chem_pos_diff == 0:
                
                prod_clear_pos = prod_pos
                prod_pos = "{}*#*{}".format(prod_left ,prod_right)
                chem_pos = "{}*#*{}".format(chem_left,chem_right)
                chem_clear_text = chem_pos_word[chem_pos]
                prodpos_chemtext[prod_pos] = chem_clear_text
            elif chem_pos_diff == 1:
                between_tokens = x.tokens[chem_right:prod_left]
                if len(chem_prod_between_tokens.intersection(set(between_tokens))) > 0:
                    prod_clear_pos = prod_pos
                    prod_pos = "{}*#*{}".format(prod_left ,prod_right)
                    chem_pos = "{}*#*{}".format(chem_left,chem_right)
                    chem_clear_text = chem_pos_word[chem_pos]
                    #print("prod_pos",prod_pos,chem_clear_text)
                    prodpos_chemtext[prod_pos] = chem_clear_text
            elif chem_pos_diff == 2:
                
                
                between_tokens = x.tokens[chem_right:prod_left]
                #print("3"*30,between_tokens)
                if len(chem_prod_between_tokens.intersection(set(between_tokens))) > 1:
                    prod_clear_pos = prod_pos
                    prod_pos = "{}*#*{}".format(prod_left ,prod_right)
                    chem_pos = "{}*#*{}".format(chem_left,chem_right)
                    chem_clear_text = chem_pos_word[chem_pos]
                    prodpos_chemtext[prod_pos] = chem_clear_text                   

    return prodpos_chemtext

In [9]:
def get_multiple_prod_chem_span(x):
    chunck_labels = {'and'}
    prod_chem_spans,_left,_right = get_prod_chem_span(x)
    #print(prod_chem_spans,_left,_right)
    multiple_prod_chem_span = {}
    span_length = len(prod_chem_spans)
    if span_length<1:
        return multiple_prod_chem_span
    else:
        for i in range(span_length-1):
            span_left = prod_chem_spans[i][1]
            span_right = prod_chem_spans[i+1][0]
            #print(span_left,span_right)
            span_diff = span_right- span_left
            if span_diff == 1:
                span_tokens = x.tokens[span_left:span_right]
                if len(chunck_labels.intersection(set(span_tokens))) > 0:
                    span_left = prod_chem_spans[i][0]
                    span_right = prod_chem_spans[i+1][1]
                    multiple_prod_chem_span[span_right] = span_left
    return multiple_prod_chem_span,_left,_right

In [10]:
def get_chem_left_edge(x,chem_word_idx):
    chem_left,chem_right = chem_word_idx
    chem_left_pos_dict = get_multiple_prod_chem_span(x)
    if chem_left_pos_dict:
        chem_left_edge = min([sub_dict.get(chem_right,chem_left)  for sub_dict in chem_left_pos_dict])
    else:
        chem_left_edge = chem_left
    
    return chem_left_edge

In [11]:
def left_right_tokens_prod(prod_word_idx,tokens,window):
    prod_word_left,prod_word_right = prod_word_idx
    #tokens = cand.tokens
    #window = 2
    prod_left_token = tokens[0:prod_word_left+1][-1 - window : -1]
    prod_right_token = tokens[prod_word_right:][0 : window]

    return prod_left_token,prod_right_token

    
def left_right_tokens_chem(chem_word_idx,tokens,window):
    chem_word_left,chem_word_right = chem_word_idx
    #tokens = cand.tokens
    #window = 2

    chem_left_token = tokens[0:chem_word_left+1][-1 - window : -1]
    chem_right_token = tokens[chem_word_right:][0 : window]

    return chem_left_token,chem_right_token

In [12]:
from collections import OrderedDict,defaultdict
def get_multiple_spans(chem_word_poss,tokens):
    """
    cluster serial numbers
    initial_dict={33: 35, 58: 59, 60: 63, 15: 16}
    result = [[(58, 59), (60, 63)]]
    
    """
    chem_pos_dig = {}
    #print("chem_word_poss",chem_word_poss)
    for chem_word_pos in chem_word_poss:
        #print()
        chem_word_start,chem_word_end = chem_word_pos.split("*#*")
        chem_start_dig = int(chem_word_start)
        chem_end_dig = int(chem_word_end)
        #chem_start_dig = get_chem_left_edge(x,[chem_start_dig,chem_end_dig])
        chem_pos_dig[chem_start_dig] = chem_end_dig
    #print(chem_pos_dig)

    new_chem_pos = OrderedDict(sorted(chem_pos_dig.items()))
    #print('new_chem_pos',new_chem_pos)
    label = 0
    record_pos_dict = defaultdict(list)
    
    for _start , _end in new_chem_pos.items():
        chem_left_tokens,chem_right_tokens = left_right_tokens_prod((_start , _end),tokens,window=1)
        #print(chem_left_tokens,chem_right_tokens)
        
        if label == 0:            
            label +=1
            tmp_end = _end
        if "from" in chem_left_tokens:
            label += 1
        elif "mol" in chem_left_tokens:
            label += 1            
        if _start - tmp_end >2:
            label +=1            
        record_pos_dict[label].append(_start)
        tmp_end = _end    
    #print('record_pos_dict',record_pos_dict)
    mutipl_spans = []
    for dig_label,dig_start in record_pos_dict.items():
        if len(dig_start)>1:
            mutipl_span = []
            for _dig_start in dig_start:
                _dig_end = chem_pos_dig[_dig_start]
                dig_start_end = (_dig_start,_dig_end)
                dig_start_end_text = '{}*#*{}'.format(_dig_start,_dig_end)
                mutipl_span.append(dig_start_end_text)
                #print(_dig_start,_dig_end)
            mutipl_spans.append(mutipl_span)
    #print("mutipl_spans","*"*20,mutipl_spans)
    return mutipl_spans

from snorkel.types import DataPoint
from snorkel.preprocess import preprocessor
@preprocessor()
def get_chemnum_between_prod_chem(cand):
    """
    Returns the chems num between prod and chem
    """
    chem_word_idx = cand.chem_word_idx
    prod_word_idx = cand.prod_word_idx
    chem_start = chem_word_idx[0]
    chem_end = chem_word_idx[1]
    prod_start = prod_word_idx[0]
    prod_end = prod_word_idx[1]
    between_chemnum = 0    
    if chem_end < prod_end:
        between_chemnum += 1
        prod_chem_order = False
        cand.between_prod_chem_chemnum = between_chemnum
        cand.prod_chem_order = prod_chem_order        
        return cand

    else:
        prod_chem_order = True
        left = prod_end
        right = chem_start
    prod_chem_nums = [i for i in range(left,right)]
    
    chem_pos_text = "{}*#*{}".format(chem_word_idx[0],chem_word_idx[1])
    cand_chems = cand.chems
    #print(cand_chems)
    cand_spans = get_multiple_spans(cand_chems,cand.tokens)
    cand_span_pos_newpos = {}
    for cand_span in cand_spans:
        for i in range(1,len(cand_span)):
            cand_span_pos_newpos[cand_span[i]] = cand_span[0] 
    #print(cand_span_pos_newpos)
    for _chem in cand.chems:
        if _chem in cand_span_pos_newpos.keys():
            _chem = cand_span_pos_newpos[_chem]
        chem_left = _chem.split("*#*")[0]
        if int(chem_left) in prod_chem_nums:
            between_chemnum += 1
    cand.between_prod_chem_chemnum = between_chemnum
    
    cand.prod_chem_order = prod_chem_order
    return cand 

In [13]:
from collections import OrderedDict,defaultdict
def get_multiple_prod_chem_spans(x,chem_word_poss,tokens):
    """
    cluster serial numbers
    initial_dict={33: 35, 58: 59, 60: 63, 15: 16}
    result = [[(58, 59), (60, 63)]]
    
    """
    chem_pos_dig = {}
    #print("chem_word_poss",chem_word_poss)
    for chem_word_pos in chem_word_poss:
        #print()
        chem_word_start,chem_word_end = chem_word_pos.split("*#*")
        chem_start_dig = int(chem_word_start)
        chem_end_dig = int(chem_word_end)
        chem_start_dig = get_chem_left_edge(x,[chem_start_dig,chem_end_dig])
        chem_pos_dig[chem_start_dig] = chem_end_dig
    #print(chem_pos_dig)

    new_chem_pos = OrderedDict(sorted(chem_pos_dig.items()))
    #print('new_chem_pos',new_chem_pos)
    label = 0
    record_pos_dict = defaultdict(list)
    
    for _start , _end in new_chem_pos.items():
        chem_left_tokens,chem_right_tokens = left_right_tokens_prod((_start , _end),tokens,window=1)
        #print(chem_left_tokens,chem_right_tokens)
        
        if label == 0:            
            label +=1
            tmp_end = _end
        if "from" in chem_left_tokens:
            label += 1
        elif "mol" in chem_left_tokens:
            label += 1            
        if _start - tmp_end >2:
            label +=1            
        record_pos_dict[label].append(_start)
        tmp_end = _end    
    #print('record_pos_dict',record_pos_dict)
    mutipl_spans = []
    for dig_label,dig_start in record_pos_dict.items():
        if len(dig_start)>1:
            mutipl_span = []
            for _dig_start in dig_start:
                _dig_end = chem_pos_dig[_dig_start]
                dig_start_end = (_dig_start,_dig_end)
                dig_start_end_text = '{}*#*{}'.format(_dig_start,_dig_end)
                mutipl_span.append(dig_start_end_text)
                #print(_dig_start,_dig_end)
            mutipl_spans.append(mutipl_span)
    #print("mutipl_spans","*"*20,mutipl_spans)
    return mutipl_spans

In [14]:
import spacy

def get_negtive_tokens_index(cand):
    sentence = cand.sentence
    #print(sentence)
    tokens = negtivenlp(sentence)
    tokens_len = len(tokens)
    tokens_index = [tok.i for tok in tokens if tok.pos_=='VERB']
    tokens_index.append(tokens_len)
    #print(tokens_index)
    negtive_start = tokens_len
    negtive_end = tokens_len
    for token in tokens:
        try:
            left_neigbor = token.nbor(-1)
        except:
            left_neigbor = token
        token_text = token.text
        token_pos = token.pos_
        left_token_text = left_neigbor.text
        token_index = token.i  
        #print(token.text,"-->",token.pos_,"-->",token.i,"-->",left_neigbor.text)
        if  token_text in ['containing','with','of','by','factors','toward','analysis','in'] and \
        left_token_text in ['buffer','medium','presence','inhibitors','inhibited','addition','fed',\
                           'salts','application','Application','environmental','combination','phase',\
                           'treated','binding','removal','tolerance','kinetic','reduced','Treatment',\
                           'treatment','cryopreserved','media']:
            negtive_start = token_index   
            #print(negtive_start)
        """
        elif token_text in ['required']:
            negtive_start = token_index
        """
    for tok_i in tokens_index:
        if tok_i > negtive_start:
            negtive_end = tok_i
            break
    return (int(negtive_start),int(negtive_end))

In [15]:
import spacy

def get_negtive_tokens_index(cand):
    sentence = cand.sentence
    #print(sentence)
    tokens = negtivenlp(sentence)
    tokens_len = len(tokens)
    tokens_verb_index = [tok.i for tok in tokens if tok.pos_=='VERB']
    tokens_verb_index.append(tokens_len)
    #print(tokens_index)
    negtive_start = tokens_len
    negtive_end = tokens_len
    negtive_index = []
    negtive_start_list = []
    for token in tokens:
        try:
            left_neigbor = token.nbor(-1)
        except:
            left_neigbor = token
        token_text = token.text
        token_pos = token.pos_
        left_token_text = left_neigbor.text
        token_index = token.i          
        #print(token.text,"-->",token.pos_,"-->",token.i,"-->",left_neigbor.text)
        if  token_text in ['containing','with','of','by','factors','toward','analysis','in','to',\
                           'concentrations','consisting'] and \
        left_token_text in ['buffer','medium','presence','inhibitors','inhibited','addition','fed',\
                           'salts','application','Application','environmental','combination','phase',\
                           'treated','binding','removal','tolerance','kinetic','reduced','Treatment',\
                           'treatment','cryopreserved','media','cultures','effects','resistant','substrate',\
                           'pretreatment','solution','use','removal','degradation']:
            #negtive_start = token_index
            negtive_start_list.append(token_index)
            #print(negtive_start)
        """
        elif token_text in ['required']:
            negtive_start = token_index
        """
    #print(negtive_start_list)
    for neg_start in negtive_start_list:
        sub_neg_index_list = []        
        for tok_i in tokens_verb_index:
            #print("tok_i",tok_i)
            if tok_i > neg_start:
                #print(neg_start,tok_i+1)
                sub_neg_index_list = [n_index for n_index in range(neg_start,tok_i+1)]
                #print(sub_neg_index_list)
                negtive_index.extend(sub_neg_index_list)
                break
        
    negtive_index = list(set(negtive_index))
    return negtive_index

In [16]:
import spacy

def get_negtive_tokens_index_one(cand):
    sentence = cand.sentence
    #print(sentence)
    tokens = negtivenlp(sentence)
    tokens_len = len(tokens)
    tokens_verb_index = [tok.i for tok in tokens if tok.pos_ in ['VERB','DET','SCONJ']]
    tokens_verb_index.append(tokens_len)
    #print(tokens_index)
    negtive_start = tokens_len
    negtive_end = tokens_len
    negtive_index = []
    negtive_start_list = []
    for token in tokens:
        try:
            left_neigbor = token.nbor(-1)
        except:
            left_neigbor = token
        token_text = token.text
        token_pos = token.pos_
        left_token_text = left_neigbor.text
        token_index = token.i  

        #print(token.text,"-->",token.pos_,"-->",token.i,"-->",left_neigbor.text)
        if  token_text in ['added','adding','conditions','by','phase','during','inhibited',':',"using","spent",\
                          'Neither','nor','when','limiting','after','through','Dissolved']:
            #negtive_start = token_index
            negtive_start_list.append(token_index)
            #print(negtive_start)
        """
        elif token_text in ['required']:
            negtive_start = token_index
        """
    #print(negtive_start_list)
    for neg_start in negtive_start_list:
        sub_neg_index_list = []        
        for tok_i in tokens_verb_index:
            #print("tok_i",tok_i)
            if tok_i > neg_start:
                #print(neg_start,tok_i+1)
                sub_neg_index_list = [n_index for n_index in range(neg_start,tok_i+1)]
                #print(sub_neg_index_list)
                negtive_index.extend(sub_neg_index_list)
                break
        
    negtive_index = list(set(negtive_index))
    return negtive_index

In [17]:
from snorkel.labeling import labeling_function
@labeling_function()
def lf_media_chem_negtive(x):
    chem_word_left,chem_word_right = x.chem_word_idx
    prod_word_left,prod_word_right = x.prod_word_idx
    negtive_span_index_list = []
    negtive_list1 = get_negtive_tokens_index(x)
    negtive_span_index_list.extend(negtive_list1)   
    negtive_list2 =  get_negtive_tokens_index_one(x)
    negtive_span_index_list.extend(negtive_list2)
    for word_right in [chem_word_right,prod_word_right]:
        if word_right in negtive_span_index_list:
            return NEGATIVE
    return ABSTAIN

In [18]:
import spacy
def get_parse_tree_chem_negtive(x):
    sentence = x.sentence
    chem_pos_word = {}
    for chem,chem_word in x.chem_pos_word.items():
        chem_start,chem_end = chem.split("*#*")
        for chem_index in range(int(chem_start),int(chem_end)):
            chem_pos_word[chem_index] = chem_word
    
    doc = treenlp(sentence)
    # negtive_chems  with mark verb
    noun_chunk_dct = {}
    chem_word_index = {}
    chem_word_root = {}
    chem_head_children = {}
    for chunk in doc.noun_chunks:
        chunk_index = chunk.root.i
        chem_word_index[chunk.root] = chunk_index
        noun_chunk_dct[chunk.root] = chunk.root.head 
        chem_head_children[chunk.root] = [str(child)for child in chunk.root.head.head.children]
        chem_word_root[chem_word] = chunk
        if chunk.root.dep_ in ['conj','appos']:
            try:
                noun_chunk_dct[chunk.root] = noun_chunk_dct[chunk.root.head]
            except:
                continue
    negtive_chems = []
    negtiv_vocubs = ['disrupted','eliminated','included','reduced','assay','exerted','prevented','altered']
    effect_vocubs = ['had']
    for chem_root,chem_vocub in noun_chunk_dct.items():
        chem_index = chem_word_index[chem_root]
        
        chem_word = chem_pos_word.get(chem_index,'')
        if str(chem_word) and str(chem_vocub) in negtiv_vocubs:
            negtive_chems.append(chem_word)
        
        if str(chem_word) and str(chem_vocub) in effect_vocubs:
            chem_children = chem_head_children[chem_root]
            if 'effect' in chem_children:
                negtive_chems.append(chem_word)       
            
    # inhibitor chems 
    noun_chunk_dct = {}
    chem_word_root = {}
    negtive_chem_index = []
    for chunk in doc.noun_chunks:
        chunk_index = chunk.root.i        
        noun_chunk_dct[chunk.root] = chunk_index
        if chunk.root.dep_ in ['conj','appos']:
            if str(chunk.root) in ['inhibitor']:
                new_chunk_index = noun_chunk_dct[chunk.root.head]
                negtive_chem_index.append(new_chunk_index)
            if str(chunk.root.head) in ['inhibitor']:
                negtive_chem_index.append(chunk_index)
    inhibitor_chems = []
    for i in negtive_chem_index:
        inhibitor_chem = chem_pos_word.get(i,'')
        if inhibitor_chem:
            inhibitor_chems.append(inhibitor_chem)    
    negtive_chems.extend(inhibitor_chems)
    negtive_chems = list(set(negtive_chems))
    return negtive_chems

In [19]:
PrecursorTokens = {'from','using','on','via','as','containing','byproduct','in','presence','adding',\
                  'consumption'}
#byproductMarker = {}
left_negtive_tokens = PrecursorTokens
right_negtive_tokens = {'consumed','conversion','substrate','did','plus','supplementation','dissociation'} #
#titers of glucaric acid increased from unmeasurable to >0.8 g/L
def get_precursors_prodchems(cand): 
    tokens = cand.tokens  
    source_chems = ['glucose','carbon','sugar','xylose','lactose','sugar','sugars','ABE','sucrose',\
                   'Glucose','galactose']
    salt_chems = ['Fe3+','Ca(2+)','NaCl','Mg(2+)','cadmium','Cu(2+)','iron','IPTG','water','N','nitrogen',\
                 'phosphorus','Ca','nitrate','CaCl2','isopropyl beta-D-thiogalactopyranoside','Tris',\
                 'zinc','Na+','Na(+)','salt','disulfide','CaSO(4)','AlCl(3)','borate','Al(3+)','phosphate','Fe(z+)',\
                 'CaCO(3)','oxygen','isopropyl-beta-d-thiogalactopyranoside','NaOH','NADPH','NaNO3','N','KCl',\
                 'phosphorous','Zn','Mn','Cu','Fe','CO','13C','NAH','Cd(2+)','inorganic phosphorus',\
                 'mixed liquor suspended solid','MLSS','OCrtZ','dihydrofolate','C-mol C-mol-1'] 
    source_chems.extend(salt_chems)    
    precursors = source_chems
    parse_tree_negtive_chems = get_parse_tree_chem_negtive(cand)
    precursors.extend(parse_tree_negtive_chems)
    chem_pos_word = cand.chem_pos_word
    sent_chems = [chem for pos,chem in chem_pos_word.items()]
    sent_chems = list(set(sent_chems))
    cand_chems = cand.chems
    cand_spans = get_multiple_prod_chem_spans(cand,cand_chems,tokens)
    cand_span_pos_newpos = {}
    for cand_span in cand_spans:
        for i in range(1,len(cand_span)):
            cand_span_pos_newpos[cand_span[i]] = cand_span[0]
    #print(cand_span_pos_newpos,"*"*30)       
    for chem in cand.chems:
        #print(chem)
        chem_word = chem_pos_word[chem]        
        if chem in cand_span_pos_newpos.keys():
            chem = cand_span_pos_newpos[chem]
            #print("chem",chem)
        chem_start,chem_end = chem.split("*#*")
        chem_left_before = int(chem_start)
        chem_right = int(chem_end)
        #print(chem_left_before,'before',chem)
        chem_left = get_chem_left_edge(cand,[chem_left_before,chem_right])
        #print(chem_left,'after')

        chem_left_tokens,chem_right_tokens = left_right_tokens_chem([chem_left,chem_right],tokens,window=1)
        #print(chem_left_tokens,chem_right_tokens)
        if len(right_negtive_tokens.intersection(set(chem_right_tokens))) > 0:
            #print("chem_word",chem_word,"chem_right_tokens",chem_right_tokens)
            precursors.append(chem_word)
        if len(PrecursorTokens.intersection(set(chem_left_tokens))) > 0:
            #print(PrecursorTokens.intersection(set(chem_left_tokens)))
            #print("chem_word",chem_word,"chem_left_tokens",chem_left_tokens,'PrecursorTokens',PrecursorTokens)
            precursors.append(chem_word)
    precursors = list(set(precursors))
    prodchems = [_chem for _chem in sent_chems if _chem not in precursors]
    cand.precursors = precursors
    cand.prodchems = prodchems
    #print("precursors",precursors,"chems",prodchems)
    return cand

In [20]:
from snorkel.labeling import labeling_function
# Check for the `chem` words appearing between the chem and the prod
#After selecting the best combination of genes, approximately 17.3 mg/L of 1,3-dihydroxy-9(10H)-acridone (DHA)
#and 26.0 mg/L of 1,3-dihydroxy-10-methylacridone (NMA) were synthesized.
PosiveLeftTokens = {'titer','titre','titers','titres','yield','yields','yielding'\
                      'produce','produced','producing','concentration','production','productivity',\
                    'obtain','obtained','achieved','generated','generate','accumulated','accumulation'\
                    'accumulate','resulting','achieving','achieve','achieved','reaching','reached',\
                    'synthesized','synthesis','leading','up','and'
                  }
SegmentTokens = {'consisting','using','in','was',',','and','from','with','per','at','were',')','(',\
                '.'}

negtive_between_tokens = {'consisting','from','on','and','consumption'}
@labeling_function(resources=dict(PosiveLeftTokens=PosiveLeftTokens,\
                                 SegmentTokens=SegmentTokens),\
                   pre=[get_chemnum_between_prod_chem])
def lf_prod_chem_between_window_chemnum(x,PosiveLeftTokens,SegmentTokens):
    x = get_precursors_prodchems(x)
    precursors = x.precursors
    chem_word = x.chem_word
    if "respectively" in x.tokens:
        return ABSTAIN
    if chem_word in precursors:
        return ABSTAIN    
    if not x.prod_chem_order:
        return ABSTAIN
    
    media_lable =lf_media_chem_negtive(x)
    if media_lable == 0:
        return ABSTAIN    

    #print()
    if x.between_prod_chem_chemnum > 0:
        return ABSTAIN
    else:
        if chem_word in precursors:
            return ABSTAIN
        prod_left_tokens, prod_right_tokens = left_right_tokens_prod(x.prod_word_idx,x.tokens,window=2)
        chem_word_left,chem_word_right = x.chem_word_idx
        #print("*"*20)
        #print(type(chem_word_right))
        #print(x.chem_word_idx)
        window = 1
        chem_right_token = x.tokens[chem_word_right:][0 : window]
        #print(chem_word_right,chem_right_token)
        if len(negtive_between_tokens.intersection(set(x.between_tokens)))>0:
            return ABSTAIN
        
        if len(PosiveLeftTokens.intersection(set(prod_left_tokens))) > 0 \
        and len(SegmentTokens.intersection(set(chem_right_token)))>0 :
            return POSITIVE
        else:
            return ABSTAIN

In [21]:
from snorkel.labeling import labeling_function
# Check for the `mark` words appearing between the chem and the prod
# 3.80 g/L of icariside D2
# achieving ~800 mg/L of jolkinol C and over 1 g/L total oxidized casbanes
ProdChemBetweenOne = {"of",')'}
@labeling_function(resources=dict(ProdChemBetweenOne=ProdChemBetweenOne),\
                  pre=[get_chemnum_between_prod_chem])
def lf_prod_chem_between_window_markword(x, ProdChemBetweenOne):
    x = get_precursors_prodchems(x)
    precursors = x.precursors
    chem_word = x.chem_word
    if chem_word in precursors:
        return ABSTAIN
    if not x.prod_chem_order:
        return ABSTAIN
    media_lable =lf_media_chem_negtive(x)
    if media_lable == 0:
        return ABSTAIN  
    #print(negtive_span_start, negtive_span_end)
    prod_left_tokens, prod_right_tokens = left_right_tokens_prod(x.prod_word_idx,x.tokens,window=2)
    chem_word_left,chem_word_right = x.chem_word_idx
   
    token_length = len(x.between_tokens)
    if token_length == 0:
        return POSITIVE
    elif token_length == 1 and len(ProdChemBetweenOne.intersection(set(x.between_tokens))) > 0:
        return POSITIVE  
    else:
        return ABSTAIN

In [22]:
from snorkel.types import DataPoint
from snorkel.preprocess import preprocessor
@preprocessor()
def get_chemnum_between_chem_prod(cand):
    """
    Returns the chems num between chem and prod
    """
    chem_word_idx = cand.chem_word_idx
    prod_word_idx = cand.prod_word_idx
    chem_start = chem_word_idx[0]
    chem_end = chem_word_idx[1]
    prod_start = prod_word_idx[0]
    prod_end = prod_word_idx[1]
    between_chemnum = 0    
    if chem_end > prod_end:
        between_chemnum += 1
        chem_prod_order = False
        cand.between_chem_prod_chemnum = between_chemnum
        cand.chem_prod_order = chem_prod_order        
        return cand
    else:
        chem_prod_order = True
        left = chem_end + 1
        right = prod_start
    chem_prod_nums = [i for i in range(left,right)]
    
    for _chem in cand.chems:
        chem_left = _chem.split("*#*")[0]
        if int(chem_left) in chem_prod_nums:
            between_chemnum += 1
    cand.between_chem_prod_chemnum = between_chemnum
    cand.chem_prod_order = chem_prod_order
    return cand            

In [23]:
"""
zingerone (70 mg/L, 0.36 mM)
"""

NegtiveBetweenToken = {'and',','}
PosiveBetweenToken = {'titer','titre','titers','titres','yield','yields',\
                      'amount','concentration','production','productivity',\
                      '(','reaching','reached','synthesized'
                  }
@labeling_function(resources=dict(PosiveBetweenToken=PosiveBetweenToken,\
                                 NegtiveBetweenToken=NegtiveBetweenToken),\
                   pre=[get_chemnum_between_chem_prod])
def lf_chem_prod_between_window_markword(x,NegtiveBetweenToken,PosiveBetweenToken):
    
    if "respectively" in x.tokens:
        return ABSTAIN 
    
    x = get_precursors_prodchems(x)
    precursors = x.precursors
    chem_word = x.chem_word
    prod_word_idx = x.prod_word_idx
   
    if chem_word in precursors:
        return ABSTAIN    

    media_lable =lf_media_chem_negtive(x)
    if media_lable == 0:
        return ABSTAIN  
    
    if not x.chem_prod_order:
        return ABSTAIN 
  
    if x.between_chem_prod_chemnum == 0:
        prod_text = "{}*#*{}".format(prod_word_idx[0],prod_word_idx[1])
        prodpos_chemtext = get_clear_prodpos_chemtext(x)
        if prodpos_chemtext:
            if prod_text in prodpos_chemtext.keys():
                clear_chem_text = prodpos_chemtext[prod_text]
                if chem_word != clear_chem_text:
                    return ABSTAIN          
        if len(NegtiveBetweenToken.intersection(set(x.between_tokens)))>0:
            #prod_left_token,prod_right_token = left_right_tokens(x)
            prod_left_token,prod_right_token = left_right_tokens_prod(x.prod_word_idx,x.tokens,window=2)
            if 'reaching' in x.between_tokens:
                return POSITIVE
            elif 'and' in prod_left_token and 'in' in prod_right_token:
                return POSITIVE
            elif ',' in prod_left_token and ')' in prod_right_token:
                return POSITIVE            
            elif ',' in x.between_tokens and 'which' in x.between_tokens and \
            len(PosiveBetweenToken.intersection(set(x.between_tokens)))>0:
                return POSITIVE
            else:
                return ABSTAIN
         
        if len(PosiveBetweenToken.intersection(set(x.between_tokens)))>0:
            return POSITIVE
        else:
            return ABSTAIN
    else:
        return ABSTAIN

In [24]:
#phenylalanine and 64 +- 3.7 mg/gDW (41 +- 2.3 mg/L) of tyrosine  
#64 +- 3.7 mg/gDW pos are replaced by 41 +- 2.3 mg/L pos
def prod_end_right_pos(candidate):
    candidate_prods = candidate.prods
    cand_span_chems=get_multiple_spans(candidate_prods,candidate.tokens)
    cand_span_pos_newpos = {}
    for cand_span_chem in cand_span_chems:
        for i in range(len(cand_span_chem)-1):
            cand_span_pos_newpos[cand_span_chem[i]] = cand_span_chem[-1]
    #print(cand_span_pos_newpos)

    return cand_span_pos_newpos

In [25]:
"""
phenylalanine and 64 +- 3.7 mg/gDW (41 +- 2.3 mg/L) of tyrosine #prod_end_right_pos
"""
NegtiveBetweenToken = {'and',','}
@labeling_function(resources=dict(NegtiveBetweenToken=NegtiveBetweenToken),\
                   pre=[get_chemnum_between_chem_prod])
def lf_chem_prod_between_window_negtiveword(x,NegtiveBetweenToken):
    if not x.chem_prod_order:
        return ABSTAIN
    x = get_precursors_prodchems(x)
    precursors = x.precursors

    #prod_chem_label = [0]
    if x.between_chem_prod_chemnum == 0:
        chem_word = x.chem_word

        prod_word_idx = x.prod_word_idx
        prod_pos = prod_end_right_pos(x)   #64 +- 3.7 mg/gDW (41 +- 2.3 mg/L) of tyrosine
        prod_pos_str = '*#*'.join([str(p_pos) for p_pos in x.prod_word_idx])
        if prod_pos_str in prod_pos.keys():
            prod_word_idx =[int(p_pos) for p_pos in prod_pos[prod_pos_str].split("*#*")]
        prod_pos_start,prod_pos_end = prod_word_idx
        chem_prod_diff = [int(chem.split("*#*")[0]) - prod_pos_end for chem in x.chems]
        #print(chem_prod_diff,x.chems,x.prods)
        
        prod_left_token,prod_right_token = left_right_tokens_prod(prod_word_idx,x.tokens,window=2)
        #print(prod_left_token,prod_right_token)
        chem_left_token,chem_right_token = left_right_tokens_chem(x.chem_word_idx,x.tokens,window=2)
        
        chem_pos_start,chem_pos_end = x.chem_word_idx
        #64.2 g/L acetoin(x.chem_word)  (prod_end, chem_start)
        prod_chem_diff = [chem_pos_start - int(prod.split("*#*")[1]) for prod in x.prods]
        #"""
        if 'from' in prod_left_token:
            return NEGATIVE
        #"""
        if chem_word in precursors:
            #print("$"*50,chem_word)
            return NEGATIVE
        
        if 0 in chem_prod_diff:
            return NEGATIVE

        elif 1 in chem_prod_diff:
            
            if 'of' in prod_right_token:
                return NEGATIVE
        elif 2 in chem_prod_diff:
            #print(prod_right_token)
            if 'of' in prod_right_token and ')' in prod_right_token:
                return NEGATIVE
        #627 +- 140 mg/L of psilocybin and 580 +- 276 mg/L of the dephosphorylated degradation product psilocin
        if 'of' in prod_right_token:
            return NEGATIVE

        if 0 in prod_chem_diff:  #156 g/L glucose  (prod_end, chem_start)
            return NEGATIVE
        elif 1 in prod_chem_diff:
            if 'of' in chem_left_token:
                #print ("&"*30,prod_chem_diff)
                return NEGATIVE
            
        if 'byproduct' in x.between_tokens:
            return NEGATIVE
        else:
            return ABSTAIN

   
    else:
        return ABSTAIN

In [26]:
PosiveLeftToken = {'titer','titre','titers','titres','yield','yields',\
                      'amount','concentration','production','productivity',\
                   'synthesis','biosynthesis','accumulation','producing','produced',\
                   'overproduced'
                  }

@labeling_function(resources=dict(PosiveLeftToken=PosiveLeftToken),
                   pre=[get_chemnum_between_chem_prod])
def lf_chem_prod_first_titer(x,PosiveLeftToken):
    x = get_precursors_prodchems(x)
    precursors = x.precursors
    chem_word = x.chem_word
    if chem_word in precursors:
        return ABSTAIN    

    media_lable =lf_media_chem_negtive(x)
    if media_lable == 0:
        return ABSTAIN  
        
    if not x.chem_prod_order:
        return ABSTAIN
    
    if x.between_chem_prod_chemnum > 0:
        return ABSTAIN
    
    chem_word_idx = x.chem_word_idx
    cand_tokens = x.tokens
    window = 2
    chem_word_neigbor_left = chem_word_idx[0]+1 - window
    chem_word_neigbor_right = chem_word_idx[1]+ window
    chem_neigbor_word=cand_tokens[chem_word_neigbor_left:chem_word_neigbor_right]
    
    chem_word_start =chem_word_idx[0] + 1
    chem_word_end = chem_word_idx[1]       
    prod_word_start = x.chem_word_idx[0]
    chem_prod_nums = [i for i in range(chem_word_end,prod_word_start)]
    between_prodnum = 0
    for _prod in x.prods:
        prod_left = _prod.split("*#*")[0]
        if int(prod_left) in chem_prod_nums:
            between_prodnum += 1

            
    prod_left_token,prod_right_token = left_right_tokens_prod(x.prod_word_idx,x.tokens,window=1)
    negtive_mark = {'substrate','consumption'}
    if len(negtive_mark.intersection(set(prod_right_token)))>0:
        return ABSTAIN            
            
            
    if len(PosiveLeftToken.intersection(set(chem_neigbor_word))) > 0 and \
    between_prodnum == 0:
        return POSITIVE
    else:
        return ABSTAIN

In [27]:
from snorkel.labeling import labeling_function

@labeling_function(pre=[get_precursors_prodchems])
def lf_one_prodchem(x):
    prodchems = x.prodchems
    #precursors= x.precursors
    chem_word = x.chem_word
    prod_word_idx = x.prod_word_idx
    if 'respectively' in x.tokens:
        return ABSTAIN
    
    prod_text = "{}*#*{}".format(prod_word_idx[0],prod_word_idx[1])
    if chem_word in prodchems and len(prodchems)==1:
        media_lable =lf_media_chem_negtive(x)
        if media_lable == 0:
            return ABSTAIN
        prod_left_token,prod_right_token = left_right_tokens_prod(prod_word_idx,x.tokens,window=1)
        negtive_mark = {'substrate'}
        if len(negtive_mark.intersection(set(prod_right_token)))>0:
            return ABSTAIN
        
        
        prodpos_chemtext = get_clear_prodpos_chemtext(x)
        if prodpos_chemtext:
            if prod_text in prodpos_chemtext.keys():
                clear_chem_text = prodpos_chemtext[prod_text]
                if chem_word == clear_chem_text:
                    return POSITIVE
                else:
                    return ABSTAIN
        return POSITIVE
    else:
        #print("i"*20)
        return ABSTAIN

In [28]:
from snorkel.labeling import labeling_function

@labeling_function(pre=[get_precursors_prodchems])
def lf_one_prodchem_negtive(x):
    #prodchems = x.prodchems
    precursors= x.precursors
    chem_word = x.chem_word
    if chem_word in precursors:
        return NEGATIVE
    else:
        return ABSTAIN

In [29]:
from snorkel.labeling import labeling_function
@labeling_function(pre=[get_precursors_prodchems])
def lf_one_productivity(x):
    prods= x.prods
    precursors= x.precursors
    prodchems = x.prodchems
    chem_word = x.chem_word
    chem_pos_word = x.chem_pos_word
    cand_chems = x.chems
    
    cand_spans = get_multiple_spans(cand_chems,x.tokens)
    #print("cand_chems:",cand_chems,"cand_spans",cand_spans,"chem_pos_word",chem_pos_word)
    
    
    chem_word = x.chem_word
    if chem_word in precursors:
        return ABSTAIN          
    media_lable =lf_media_chem_negtive(x)
    if media_lable == 0:
        return ABSTAIN    

    chem_words = []
    if len(prods) > 1:
        return ABSTAIN
    if len(cand_spans) >0:
                
        for cand_span in cand_spans:
            if len(cand_span) < 2:
                return ABSTAIN            
            for chem_span in cand_span:
                sub_chem_word = chem_pos_word[chem_span]
                if sub_chem_word in prodchems:
                    chem_words.append(sub_chem_word)
    else:
        return ABSTAIN
    #print(chem_words,chem_word)        
    if chem_word in chem_words:
        prod_word_idx = x.prod_word_idx
        prod_text = "{}*#*{}".format(prod_word_idx[0],prod_word_idx[1])
        prodpos_chemtext = get_clear_prodpos_chemtext(x)
        if prodpos_chemtext:
            if prod_text in prodpos_chemtext.keys():
                clear_chem_text = prodpos_chemtext[prod_text]
                if chem_word != clear_chem_text:
                    return ABSTAIN
        prod_left_token,prod_right_token = left_right_tokens_prod(prod_word_idx,x.tokens,window=1)

        chem_word_idx = x.chem_word_idx
        positive_mark = {'of','-'}
        chem_prod_diff = chem_word_idx[0] - prod_word_idx[1]
        if len(positive_mark.intersection(set(prod_right_token)))>0 and chem_prod_diff!= 1:
            return ABSTAIN
        negtive_mark = {'or'}
        chem_left_token,chem_right_token = left_right_tokens_chem(chem_word_idx,x.tokens,window=1)        
        if len(negtive_mark.intersection(set(chem_left_token)))>0:
            return ABSTAIN        
        return POSITIVE
    else:
        
        return ABSTAIN

In [30]:
from snorkel.labeling import labeling_function

@labeling_function(pre=[get_precursors_prodchems])
def lf_one_productivity_negtive(x):
    prods= x.prods
    precursors= x.precursors
    #prodchems = x.prodchems
    chem_word = x.chem_word
    chem_pos_word = x.chem_pos_word
    cand_chems = x.chems
    cand_spans = get_multiple_spans(cand_chems,x.tokens)
    #print("cand_spans",cand_spans)
    precursors_chem_words = []
    if len(prods) > 1:
        return ABSTAIN
    if len(cand_spans) >0:
        for cand_span in cand_spans:
            if len(cand_span) < 2:
                return ABSTAIN            
            for chem_span in cand_span:
                sub_chem_word = chem_pos_word[chem_span]
                #if sub_chem_word in prodchems:
                if sub_chem_word in precursors:
                    precursors_chem_words.append(sub_chem_word)
    else:
        return ABSTAIN
    #print(precursors_chem_words,chem_word)        
    if chem_word in precursors_chem_words:
        return NEGATIVE
    else:
        #print("i"*20)
        return ABSTAIN 

In [31]:
def get_mutiple_chem_prod_match(cand):
    chem_word_poss = cand.chems
    prod_word_poss = cand.prods
    #chem_tokens_num = len(cand.tokens)
    chem_word_span = get_multiple_spans(chem_word_poss,cand.tokens)
    prod_word_span = get_multiple_spans(prod_word_poss,cand.tokens)
    #print("chem_word_span",chem_word_span)
    #print("prod_word_span",prod_word_span)
    chem_prod_matches = []
    for sub_chem_spans in chem_word_span:
        for sub_prod_spans in prod_word_span:
            if len(sub_prod_spans) != len(sub_chem_spans):
                continue
            for sub_chem_word,sub_prod_word in zip(sub_chem_spans,sub_prod_spans):
                #print(sub_chem_word,sub_prod_word)
                chem_prod_match = "{}#*#{}".format(sub_chem_word,sub_prod_word)
                #print(chem_prod_match)
                chem_prod_matches.append(chem_prod_match)
    #print(chem_prod_matches)
    cand.chem_prod_matches = chem_prod_matches
    return cand

In [32]:
from snorkel.labeling import labeling_function
multiple_tokens = {'and',','}
@labeling_function( resources=dict(multiple_tokens=multiple_tokens),\
                   pre=[get_mutiple_chem_prod_match])
def lf_chem_prod_match(cand,multiple_tokens):
    cand_tokens = cand.tokens
    if "respectively" in cand_tokens:
        cand = get_precursors_prodchems(cand)
        precursors= cand.precursors
        chem_word = cand.chem_word
        if chem_word in precursors:
            return ABSTAIN          
        media_lable =lf_media_chem_negtive(cand)
        if media_lable == 0:
            return ABSTAIN  
        
        
        chem_word_idx = cand.chem_word_idx
        prod_word_idx = cand.prod_word_idx
        if chem_word_idx[0] > prod_word_idx[0]:
            return ABSTAIN
        window = 2
        chem_word_neigbor_left = chem_word_idx[0]- window
        chem_word_neigbor_right = chem_word_idx[1]+ window
        chem_neigbor_word=cand_tokens[chem_word_neigbor_left:chem_word_neigbor_right]
        if len(multiple_tokens.intersection(set(chem_neigbor_word))) == 0:
            return ABSTAIN
        #print(chem_neigbor_word)
        chem_poss = '{}*#*{}'.format(chem_word_idx[0],chem_word_idx[1])
        prod_poss = '{}*#*{}'.format(prod_word_idx[0],prod_word_idx[1])
        chem_prod_poss = "{}#*#{}".format(chem_poss,prod_poss)
        if chem_prod_poss in cand.chem_prod_matches:
            return POSITIVE
        else:
            return ABSTAIN        
    else:
        return ABSTAIN

In [33]:
from snorkel.labeling import labeling_function
multiple_tokens = {'and',','}
@labeling_function( resources=dict(multiple_tokens=multiple_tokens),\
                   pre=[get_mutiple_chem_prod_match])
def lf_chem_prod_not_match(cand,multiple_tokens):
    cand_tokens = cand.tokens
    if "respectively" in cand_tokens:
        chem_word_idx = cand.chem_word_idx
        prod_word_idx = cand.prod_word_idx
        if chem_word_idx[0] > prod_word_idx[0]:
            return ABSTAIN
        window = 2
        chem_word_neigbor_left = chem_word_idx[0]- window
        chem_word_neigbor_right = chem_word_idx[1]+ window
        chem_neigbor_word=cand_tokens[chem_word_neigbor_left:chem_word_neigbor_right]
        if len(multiple_tokens.intersection(set(chem_neigbor_word))) == 0:
            return ABSTAIN
        #print(chem_neigbor_word)
        chem_poss = '{}*#*{}'.format(chem_word_idx[0],chem_word_idx[1])
        prod_poss = '{}*#*{}'.format(prod_word_idx[0],prod_word_idx[1])
        chem_prod_poss = "{}#*#{}".format(chem_poss,prod_poss)
        if not cand.chem_prod_matches:
            return ABSTAIN 
        if chem_prod_poss in cand.chem_prod_matches:
            return ABSTAIN
        else:
            return NEGATIVE        
    else:
        return ABSTAIN

In [34]:
from snorkel.labeling import PandasLFApplier

lfs = [
    lf_prod_chem_between_window_chemnum,
    lf_prod_chem_between_window_markword,
    lf_chem_prod_between_window_markword,
    lf_chem_prod_between_window_negtiveword,
    lf_chem_prod_first_titer,
    lf_one_prodchem,
    lf_one_prodchem_negtive,
    lf_media_chem_negtive,
    lf_one_productivity,
    lf_one_productivity_negtive,
    lf_chem_prod_match,
    lf_chem_prod_not_match,

]

applier = PandasLFApplier(lfs)

In [35]:
from snorkel.labeling import LFAnalysis

negtivenlp = spacy.load("en_core_web_sm")
L_dev = applier.apply(df_dev)
L_train = applier.apply(df_dev)


/Users/tianyu/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 2063/2063 [05:03<00:00,  6.79it/s]


In [43]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, Y_dev, n_epochs=5000, log_freq=500, seed=12345)

In [44]:
from snorkel.analysis import metric_score
from snorkel.utils import probs_to_preds

probs_dev = label_model.predict_proba(L_dev)
preds_dev = probs_to_preds(probs_dev)
print(
    f"Label model f1 score: {metric_score(Y_dev, preds_dev, probs=probs_dev, metric='f1')}"
)
print(
    f"Label model roc-auc: {metric_score(Y_dev, preds_dev, probs=probs_dev, metric='roc_auc')}"
)

Label model f1 score: 0.781582054309327
Label model roc-auc: 0.9125938996068865


In [45]:
label_model.save('../ProcessedData/procon_saved_label_model.pkl')